Advanced Features
---

By now, you’ve learned the most commonly used parts of the Rust programming  
language. Before we do one more project in Chapter 20, we’ll look at a few  
aspects of the language you might run into every once in a while, but may not  
use every day. You can use this chapter as a reference for when you encounter  
any unknowns. The features covered here are useful in very specific situations.  
Although you might not reach for them often, we want to make sure you have a  
grasp of all the features Rust has to offer.

In this chapter, we’ll cover:

- **`Unsafe Rust:`** how to opt out of some of Rust’s guarantees and take  
responsibility for manually upholding those guarantees
- **`Advanced traits:`** associated types, default type parameters, fully  
qualified syntax, supertraits, and the newtype pattern in relation to traits
- **`Advanced types:`** more about the newtype pattern, type aliases, the never  
type, and dynamically sized types
- **`Advanced functions and closures:`** function pointers and returning  
closures  
- **`Macros:`** ways to define code that defines more code at compile time  

It’s a panoply of Rust features with something for everyone! Let’s dive in!

### Unsafe Rust

All the code we’ve discussed so far has had Rust’s memory safety guarantees  
enforced at compile time. However, Rust has a second language hidden inside it  
that doesn’t enforce these memory safety guarantees:  

- it’s called unsafe Rust and works just like regular Rust, but gives us extra  
superpowers.

**`Unsafe Rust`** exists because, by nature, static analysis is conservative.  
When the compiler tries to determine whether or not code upholds the guarantees,  
it’s better for it to reject some valid programs than to accept some invalid  
programs. Although the code might be okay, if the Rust compiler doesn’t have  
enough information to be confident, it will reject the code. In these cases,  
you can use unsafe code to tell the compiler, “Trust me, I know what I’m doing.”  
Be warned, however, that you use unsafe Rust at your own risk:  

- if you use unsafe code incorrectly, problems can occur due to memory unsafety,  
such as null pointer dereferencing.

Another reason Rust has an unsafe alter ego is that the underlying computer  
hardware is inherently unsafe. If Rust didn’t let you do unsafe operations, you  
couldn’t do certain tasks. Rust needs to allow you to do low-level systems  
programming, such as directly interacting with the operating system or even  
writing your own operating system. Working with low-level systems programming is  
one of the goals of the language. Let’s explore what we can do with unsafe Rust  
and how to do it.

### Unsafe SuperPowers

To switch to unsafe Rust, use the **`unsafe`** keyword and then start a new  
block that holds the unsafe code. You can take five actions in unsafe Rust that  
you can’t in safe Rust, which we call unsafe superpowers. Those superpowers  
include the ability to:

- Dereference a raw pointer
- Call an unsafe function or method
- Access or modify a mutable static variable
- Implement an unsafe trait
- Access fields of `union`s

It’s important to understand that unsafe doesn’t turn off the borrow checker or  
disable any other of Rust’s safety checks:  
- if you use a reference in unsafe code, it will still be checked.  
The unsafe keyword only gives you access to these five features that are then  
not checked by the compiler for memory safety. You’ll still get some degree of  
safety inside of an unsafe block.

In addition, unsafe does not mean the code inside the block is necessarily  
dangerous or that it will definitely have memory safety problems:  

- the intent is that as the programmer, you’ll ensure the code inside an unsafe  
block will access memory in a valid way.

People are fallible, and mistakes will happen, but by requiring these five  
unsafe operations to be inside blocks annotated with unsafe you’ll know that  
any errors related to memory safety must be within an unsafe block. Keep unsafe  
blocks small; you’ll be thankful later when you investigate memory bugs.

To isolate unsafe code as much as possible, it’s best to enclose unsafe code  
within a safe abstraction and provide a safe API, which we’ll discuss later in  
the chapter when we examine unsafe functions and methods. Parts of the standard  
library are implemented as safe abstractions over unsafe code that has been  
audited. Wrapping unsafe code in a safe abstraction prevents uses of unsafe  
from leaking out into all the places that you or your users might want to use  
the functionality implemented with unsafe code, because using a safe abstraction  
is safe.

Let’s look at each of the five unsafe superpowers in turn. We’ll also look at  
some abstractions that provide a safe interface to unsafe code.

### Dereferencing a Raw Pointer

In Chapter 4, in the “Dangling References” section, we mentioned that the  
compiler ensures references are always valid. Unsafe Rust has two new types  
called raw pointers that are similar to references. As with references, raw  
pointers can be immutable or mutable and are written as *const T and *mut T,  
respectively. The asterisk isn’t the dereference operator; it’s part of the  
type name. In the context of raw pointers, immutable means that the pointer  
can’t be directly assigned to after being dereferenced.

Different from references and smart pointers, raw pointers:

- Are allowed to ignore the borrowing rules by having both immutable and  
mutable pointers or multiple mutable pointers to the same location
- Aren’t guaranteed to point to valid memory
- Are allowed to be null
- Don’t implement any automatic cleanup

By opting out of having Rust enforce these guarantees, you can give up  
guaranteed safety in exchange for greater performance or the ability to  
interface with another language or hardware where Rust’s guarantees don’t  
apply.

> Listing 19-1 shows how to create an immutable and a mutable raw pointer  
from references.

In [2]:
{
    let mut num = 5;

    let r1 = &num as *const i32;
    let r2 = &mut num as *mut i32;
}

()

> Listing 19-1: Creating raw pointers from references

Notice that we don’t include the unsafe keyword in this code. We can create raw  
pointers in safe code; we just can’t dereference raw pointers outside an unsafe  
block, as you’ll see in a bit.

We’ve created raw pointers by using as to cast an immutable and a mutable  
reference into their corresponding raw pointer types. Because we created them  
directly from references guaranteed to be valid, we know these particular raw  
pointers are valid, but we can’t make that assumption about just any raw pointer.

To demonstrate this, next we’ll create a raw pointer whose validity we can’t be  
so certain of. Listing 19-2 shows how to create a raw pointer to an arbitrary  
location in memory. Trying to use arbitrary memory is undefined: there might be  
data at that address or there might not, the compiler might optimize the code  
so there is no memory access, or the program might error with a segmentation  
fault. Usually, there is no good reason to write code like this, but it is  
possible.

In [3]:
{
    let address = 0x012345usize;
    let r = address as *const i32;
}

()

> Listing 19-2: Creating a raw pointer to an arbitrary memory address

Recall that we can create raw pointers in safe code, but we can’t dereference  
raw pointers and read the data being pointed to. In Listing 19-3, we use the  
dereference operator * on a raw pointer that requires an unsafe block.

In [6]:
{
    let mut num = 5;

    let r1 = &num as *const i32;
    let r2 = &mut num as *mut i32;

    unsafe {
        // Creating a pointer does no harm @audit : why???
        // It's only when we try to access the value that it points at that we
        // might end up dealing with an invalid value
        println!("r1 is {}", *r1);
        println!("r2 is {}", *r2);
    }
}

r1 is 5
r2 is 5


()

> Listing 19-3: Dereferencing raw pointers within an unsafe block

Creating a pointer does no harm; it’s only when we try to access the value that  
it points at that we might end up dealing with an invalid value.

Note also that in Listing 19-1 and 19-3, we created *const i32 and *mut i32 raw  
pointers that both pointed to the same memory location, where num is stored. If  
we instead tried to create an immutable and a mutable reference to num, the code  
would not have compiled because Rust’s ownership rules don’t allow a mutable  
reference at the same time as any immutable references. With raw pointers, we  
can create a mutable pointer and an immutable pointer to the same location and  
change data through the mutable pointer, potentially creating a data race.  
Be careful!

With all of these dangers, why would you ever use raw pointers? One major use  
case is when interfacing with C code, as you’ll see in the next section,  
“Calling an Unsafe Function or Method.” Another case is when building up safe  
abstractions that the borrow checker doesn’t understand. We’ll introduce unsafe  
functions and then look at an example of a safe abstraction that uses unsafe  
code.

### Calling an Unsafe Function or Method

The second type of operation you can perform in an unsafe block is calling  
unsafe functions. Unsafe functions and methods look exactly like regular  
functions and methods, but they have an extra unsafe before the rest of the  
definition. The unsafe keyword in this context indicates the function has  
requirements we need to uphold when we call this function, because Rust can’t  
guarantee we’ve met these requirements. By calling an unsafe function within an  
unsafe block, we’re saying that we’ve read this function’s documentation and  
take responsibility for upholding the function’s contracts.

Here is an unsafe function named dangerous that doesn’t do anything in its  
body:

In [13]:
unsafe fn dangerous() {}

// this unsafe block is required to call an unsafe function
unsafe {
    dangerous();
}

()

We must call the dangerous function within a separate unsafe block. If we try  
to call dangerous without the unsafe block, we’ll get an error:

```sh
$ cargo run
   Compiling unsafe-example v0.1.0 (file:///projects/unsafe-example)
error[E0133]: call to unsafe function is unsafe and requires unsafe function or block
 --> src/main.rs:4:5
  |
4 |     dangerous();
  |     ^^^^^^^^^^^ call to unsafe function
  |
  = note: consult the function's documentation for information on how to avoid undefined behavior

For more information about this error, try `rustc --explain E0133`.
error: could not compile `unsafe-example` due to previous error
```

With the unsafe block, we’re asserting to Rust that we’ve read the function’s  
documentation, we understand how to use it properly, and we’ve verified that  
we’re fulfilling the contract of the function.

Bodies of unsafe functions are effectively unsafe blocks, so to perform other  
unsafe operations within an unsafe function, we don’t need to add another  
unsafe block.

### Creating a Safe Abstraction over Unsafe Code

Just because a function contains unsafe code doesn’t mean we need to mark the  
entire function as unsafe. In fact, wrapping unsafe code in a safe function is  
a common abstraction. As an example, let’s study the split_at_mut function from  
the standard library, which requires some unsafe code. We’ll explore how we  
might implement it. 

This safe method is defined on mutable slices:  
- it takes one slice and makes it two by splitting the slice at the index given  
as an argument. 

> Listing 19-4 shows how to use split_at_mut.

In [20]:
{
    let mut v = vec![1, 2, 3, 4, 5, 6];

    let r = &mut v[..];

    let (a, b) = r.split_at_mut(3);

    assert_eq!(a, &mut [1, 2, 3]);
    assert_eq!(b, &mut [4, 5, 6]);
}

()

> Listing 19-4: Using the safe split_at_mut function

We can’t implement this function using only safe Rust. An attempt might look  
something like Listing 19-5, which won’t compile. For simplicity, we’ll  
implement split_at_mut as a function rather than a method and only for slices  
of i32 values rather than for a generic type T.

In [35]:
/// Takes a slice
/// - split at mid
/// - returns a tuple of the resulting 2 slices
fn split_at_mut(values: &mut [i32], mid: usize) -> (&mut [i32], &mut [i32]){
    let len = values.len();

    // safety check that `mid` is within our slice bounds
    // - else, PANIC and TERMINATE program HERE
    assert!(mid <= len);

    // @audit-ok : rust borrow checker will not allow this because `values` is 
    // mutable
    // - and can ONLY be borrowed ONCE
    // - in generating the 2nd slice, `values` is borrowed AGAIN
    (&mut values[..mid], &mut values[mid..])
    // Borrowing different NON OVERLAPPING parts of a slice is FUNDAMENTALLY OK
    // @audit-ok ... but Rust borrow checker DOES NOT KNOW THIS
    // - this is a SUFFICIENT ENOUGH basis to deploy `unsafe` code
}

Error: cannot borrow `*values` as mutable more than once at a time

> Listing 19-5: An attempted implementation of split_at_mut using only safe Rust

This function first gets the total length of the slice. Then it asserts that  
the index given as a parameter is within the slice by checking whether it’s  
less than or equal to the length. The assertion means that if we pass an index  
that is greater than the length to split the slice at, the function will panic  
before it attempts to use that index.

Then we return two mutable slices in a tuple: 

- one from the start of the original slice to the mid index  
- and another from mid to the end of the slice.

When we try to compile the code in Listing 19-5, we’ll get an error.

```sh
$ cargo run
   Compiling unsafe-example v0.1.0 (file:///projects/unsafe-example)
error[E0499]: cannot borrow `*values` as mutable more than once at a time
 --> src/main.rs:6:31
  |
1 | fn split_at_mut(values: &mut [i32], mid: usize) -> (&mut [i32], &mut [i32]) {
  |                         - let's call the lifetime of this reference `'1`
...
6 |     (&mut values[..mid], &mut values[mid..])
  |     --------------------------^^^^^^--------
  |     |     |                   |
  |     |     |                   second mutable borrow occurs here
  |     |     first mutable borrow occurs here
  |     returning this value requires that `*values` is borrowed for `'1`

For more information about this error, try `rustc --explain E0499`.
error: could not compile `unsafe-example` due to previous error
```

Rust’s borrow checker can’t understand that we’re borrowing different parts of  
the slice; it only knows that we’re borrowing from the same slice twice.  

- Borrowing different parts of a slice is fundamentally okay  
- because the two slices aren’t overlapping,  
- but Rust isn’t smart enough to know this.  

When we know code is okay, but Rust doesn’t, it’s time to reach for unsafe code.

Listing 19-6 shows how to use an unsafe block, a raw pointer, and some calls to  
unsafe functions to make the implementation of split_at_mut work.

In [32]:
// import slice module from standard library
// - provides functions for handling slices
use std::slice;

/// Takes a slice : 
/// - splits at mid
/// - returns a tuple of those two slices
fn split_at_mut(value: &mut [i32], mid: usize) -> (&mut [i32], &mut [i32]){

    // get original slice length
    let len = value.len();
    // get RAW POINTER to the data in the slice
    // - RAW POINTER allow for MORE CONTROL and LESS SAFETY in Rust
    let ptr = value.as_mut_ptr();

    // safety check that mid is within the slice bound
    // - if out of bound, PANIC and TERMINATE the program
    assert!(mid <= len);

    // start an unsafe block
    unsafe {
        // returns a tuple of two mutable slices
        (
            // from_raw_parts_mut : creates a mutable slice from :
            // - ptr : a raw pointer to the first element in the slice
            // - the length of the slice : mid
            slice::from_raw_parts_mut(ptr, mid),
            // from_raw_parts_mut : creates a mutable slice from :
            // - ptr : a raw pointer to starting point of ([0] + mid)
            // - the length of the slice : (len - mid)
            slice::from_raw_parts_mut(ptr.add(mid), len -mid)
        )
    }
}